In [1]:
from firecloud import fiss
import pandas as pd
pd.set_option('display.max_row', 10000)
import io
import numpy as np
from pprint import pprint

In [2]:
control = fiss.fapi.get_workspace_acl("topmed-shared", "topmed-shared").json()#['acl']
print control.keys()

[u'acl']


In [3]:
ent_types = fiss.fapi.list_entity_types("topmed-shared", "topmed-shared").json()
for t in ent_types.keys():
    print t, "count:", ent_types[t]['count']

sample count: 56436
participant count: 56436
sample_set count: 2


In [4]:
samples = fiss.fapi.get_entities("topmed-shared","topmed-shared", "sample").json()
sample_tups = [(samples[i]['attributes']['participant']['entityName'],samples[i]['attributes']['study']) for i in xrange(0,len(samples))]
pid_to_study = dict()
for e in sample_tups:
    pid_to_study.setdefault(e[1], []).append(e[0])

    num_dict = {d:len(pid_to_study[d]) for d in pid_to_study}
print 'Included studies(# participants):\n', '\n'.join([d+'('+str(num_dict[d])+')' for d in num_dict])

Included studies(# participants):
Partners(111)
EOCOPD(66)
SAGE(452)
JHS(3136)
CRA(1043)
GenSalt(1695)
FHS(3758)
GALAII(914)
ARIC(3616)
Mayo(1251)
GeneSTAR(1639)
WHI(10047)
VU_AF(1024)
THRV(1937)
COPDGene(8742)
WGHS(99)
BAGS(968)
HyperGEN(1662)
GOLDN(904)
SAS(1208)
CHS(70)
MESA(4880)
GENOA(1144)
DHS(339)
SAFS(1509)
CCAF(327)
HVH(66)
Amish(1030)
Sarcoidosis(608)
VAFAR(111)
NA(432)
MGH_AF(725)
CFS(923)


In [6]:
## start hail context
from hail import *
hc = HailContext(sc)

Running on Apache Spark version 2.0.2
SparkUI available at http://10.128.0.13:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.1-6f85985


In [7]:
samples_sets = fiss.fapi.get_entities('topmed-shared','topmed-shared', 'sample_set').json()
print 'Sample set name:', samples_sets[0]['name']
print 'Sample set fields:', ', '.join(samples_sets[0]['attributes'].keys())
vcf_files = samples_sets[0]['attributes']['vcf']['items']
print '# of vcf files:', len(vcf_files)

Sample set name: freeze5b_minDP10
Sample set fields: wgsa_subset, gds, vcf, bcf, tbi, cov_grm, samples, wgsa_raw, cor_grm
# of vcf files: 23


In [11]:
print vcf_files[1:2]

[u'gs://fc-adaae650-a458-4c56-8a55-d96fa463a5c6/25c6cc4a-9d07-49bc-a548-9166f7a5f5ad/w/e111b73f-4bff-4079-8c73-f9951d8dcb7a/call-runVcf/shard-1/freeze.5b.chr10.pass_and_fail.gtonly.minDP10.vcf.bgz']


In [13]:
vds = hc.import_vcf(vcf_files[1:2],min_partitions = 1000).split_multi()

2017-11-22 22:00:15 Hail: INFO: No multiallelics detected.
2017-11-22 22:00:16 Hail: INFO: Coerced sorted dataset
2017-11-22 22:00:16 Hail: WARN: called redundant split on an already split VDS


In [ ]:
 vds.write("gs://fc-adaae650-a458-4c56-8a55-d96fa463a5c6/vds_topmed/freeze.5b.chr10.pass_and_fail.gtonly.minDP10.vds")

In [ ]:
vds = hc.read("gs://fc-adaae650-a458-4c56-8a55-d96fa463a5c6/vds_topmed/freeze.5b.chr10.pass_and_fail.gtonly.minDP10.vds")

In [8]:
data_model = fiss.fapi.get_entities_tsv("topmed-shared","topmed-shared", "sample")
data_model_text = pd.read_csv(io.StringIO(data_model.text), sep='\t')[['entity:sample_id','participant','CENTER','study','topmed_project']]
data_model_text.rename(columns = {'entity:sample_id':'ent_sample_id', 'participant':'sample_id'}, inplace = True)
data_model_text[['study', 'topmed_project']] = data_model_text[['study', 'topmed_project']].astype(str)
data_model_text.info()

from pyspark.sql import SQLContext
sqlctx = SQLContext(hc.sc)
spark_df = sqlctx.createDataFrame(data_model_text)
kt = KeyTable.from_dataframe(spark_df,key='sample_id') 
vds = vds.annotate_samples_table(kt, root='sa')

data_model_text.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56436 entries, 0 to 56435
Data columns (total 5 columns):
ent_sample_id     56436 non-null object
sample_id         56436 non-null object
CENTER            56436 non-null object
study             56436 non-null object
topmed_project    56436 non-null object
dtypes: object(5)

NameError: name 'vds' is not defined